In [1]:
import os
import pandas as pd

In [2]:
# Change REP_DIR to the folder containing all repetitions for a block (network condition)
REPS_DIR = "wifi"
os.chdir(REPS_DIR)

In [3]:
cwd = os.path.abspath('') 
cwd

'D:\\Master\\Sem5\\VU-green-it\\experiment-data\\wifi'

In [4]:
repetition_files = os.listdir(cwd) 
repetition_files

['2020.10.11_192407',
 '2020.10.15_020636',
 '2020.10.15_032742',
 '2020.10.15_044814',
 '2020.10.15_060851',
 '2020.10.15_072920',
 '2020.10.15_084950',
 '2020.10.17_174337',
 '2020.10.17_190421',
 '2020.10.17_202505',
 '2020.10.17_214543']

In [5]:
assert(11 == len(repetition_files)) # check that no of repetitions is 11

In [6]:
def get_aggregated_rep_df(rep_dir):
    os.chdir(f"{cwd}\\{rep_dir}")
    
    # Read data from file 'Aggregated_Results_Batterystats.csv' 
    aggregated_battery_stats = pd.read_csv('Aggregated_Results_Batterystats.csv') 
    
    navigation_timing_df = pd.DataFrame(columns=['headerSize', 'workerTime', 'fetchTime', 
                                            'timeToFirstByte', 'downloadTime', 'totalTime',
                                            'totalLoadTime', 'dnsLookupTime']) # relevant
    fcp_df = pd.DataFrame(columns=['fcp']) # relevant
    fp_df = pd.DataFrame(columns=['fp']) # relevant
    
    for i in range (0, len(aggregated_battery_stats)):
        website = aggregated_battery_stats['subject'].get(i)
        
        perfumejs_path = f"{cwd}\\{rep_dir}\\data\\nexus5x\\{website}\\chrome\\perfume_js\\"
    
        for file in os.listdir(perfumejs_path):
            if os.path.isfile(os.path.join(perfumejs_path, file)):
                if 'navigationTiming_results_' in file:
                    navigation_timing_stats_for_website = pd.read_csv(f"{perfumejs_path}{file}")
                    navigation_timing_df = navigation_timing_df.append(navigation_timing_stats_for_website[:1], sort=False, ignore_index=True)

                if 'fcp_results_' in file:
                    fcp_for_website = pd.read_csv(f"{perfumejs_path}{file}")
                    fcp_df = fcp_df.append(fcp_for_website[:1], sort=False, ignore_index=True)

                if 'fp_results_' in file:
                    fp_for_website = pd.read_csv(f"{perfumejs_path}{file}")
                    fp_df = fp_df.append(fp_for_website[:1], sort=False, ignore_index=True)

    aggregated_rep = pd.concat([aggregated_battery_stats, navigation_timing_df, fcp_df, fp_df], join = 'outer', axis = 1) 
    assert(30==len(aggregated_rep))

    aggregated_rep.to_csv(f"{cwd}\\{rep_dir}\\Aggregated_Results_For_Rep.csv")

    return aggregated_rep

In [7]:
all_aggregated_reps = pd.DataFrame()

i = 0
for rep_dir in repetition_files:
    aggregated_rep_df = get_aggregated_rep_df(rep_dir)
    
    assert(len(aggregated_rep_df) == 30)
    
    # Add repetition timestamp (corresponds to rep folder name)
    rep_name_df = pd.concat([pd.DataFrame([rep_dir], columns=['timestamp']) for i in range(30)], ignore_index=True)
    
    aggregated_rep_df = pd.concat([aggregated_rep_df, rep_name_df], axis = 1)
    
    # Aggregate repetition df to result df
    old_len = len(all_aggregated_reps)
    all_aggregated_reps = pd.concat([all_aggregated_reps, aggregated_rep_df])
    new_len = len(all_aggregated_reps)
    
    assert(len(all_aggregated_reps) - old_len == 30)

In [8]:
assert(len(all_aggregated_reps) == 11*30) # 11 repetitions, 30 websites (360)

In [9]:
all_aggregated_reps.head()

,device,subject,browser,batterystats_Joule_calculated,headerSize,workerTime,fetchTime,timeToFirstByte,downloadTime,totalTime,totalLoadTime,dnsLookupTime,fcp,fp,timestamp
0,nexus5x,http-192-168-100-103-9191-local_websites-www-i...,chrome,49.743360,341,0,153.4,3.9,135.2,139.1,5018,0,1472.3,1472.3,2020.10.11_192407
1,nexus5x,http-192-168-100-103-9191-local_websites-www-w...,chrome,64.089072,341,0,76.5,4.5,42.5,47,1842,0,1282,1282,2020.10.11_192407
2,nexus5x,http-192-168-100-103-9191-local_websites-www-w...,chrome,61.712856,334,0,88.7,13.7,49,62.7,1008,0,1172.7,1172.7,2020.10.11_192407
3,nexus5x,http-192-168-100-103-9191-local_websites-www-o...,chrome,49.466808,342,0,40.5,4.2,3.9,8.1,2730,0,1787.4,1787.4,2020.10.11_192407
4,nexus5x,http-192-168-100-103-9191-local_websites-www-b...,chrome,43.525440,346,0,143.6,34.8,80.5,115.3,1015,0,1185.4,1185.4,2020.10.11_192407


In [10]:
all_aggregated_reps.to_csv(f"{cwd}\\Aggregated_Reps_Wifi.csv")